<a href="https://colab.research.google.com/github/dipta007/hate-speech-election-2020/blob/main/Hate_Speech_distilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(
          "mrm8488/distilroberta-finetuned-tweets-hate-speech",
          # num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
          output_attentions = False, # Whether the model returns attentions weights.
          output_hidden_states = False, # Whether the model returns all hidden-states.
        )

In [6]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [7]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-tweets-hate-speech")

In [8]:
for param in model.roberta.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    print(param.requires_grad)

True
True
True
True


In [9]:
from datasets import load_dataset

dataset = load_dataset("hatexplain")

Reusing dataset hatexplain (/root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/802fcd855438812094e336cea509c99b04b890e4e0846c0385877ee2c7361e93)


In [10]:
" ".join(dataset["train"][0]["post_tokens"])

'u really think i would not have been raped by feral hindu or muslim back in india or bangladesh and a neo nazi would rape me as well just to see me cry'

In [11]:
dataset["train"][0]["annotators"]["label"]

[0, 2, 2]

In [12]:
for i in range(40, 50):
  print(dataset["train"][i]["annotators"]["label"])
  print(" ".join(dataset["train"][i]["post_tokens"]))
  print("")

[0, 0, 0]
a nigger bich will never have strait hair so keep wearing dat horsehair weave

[0, 0, 0]
apparently some vile nigger animals kidnapped a white special needs teen and tortured and scalped him while shouting fuck white people fuck trump are you being armed and are you shooting in the head when these zombies approach

[0, 0, 0]
<user> you need potassium eat a banana you fucking nigger

[2, 0, 0]
well then send some coon estrogen to my wife she turning more liberal breath by breath due to fake news i do not know how much more i can take hell between her and my two daughters hormones i have started growing breast tissue like pseudo naturally becoming bruce jenner

[0, 0, 0]
it high time we hindus should think that these filthy muslims should be allowed to live here or not kamleshtiwari ypg yourprophetisgay

[1, 1, 1]
<user> why should you ban immigrants from operating small business every business start somewhere and tomorrow that will give people job

[2, 0, 2]
i am sure you are 

In [13]:
def join_all(arr):
  ret = []
  for v in arr:
    ret.append(" ".join(v))
  return ret

train_encodings = tokenizer(join_all(dataset["train"]["post_tokens"]), padding=True, truncation=True, return_tensors="pt")
val_encodings = tokenizer(join_all(dataset["validation"]["post_tokens"]), padding=True, truncation=True, return_tensors="pt")
test_encodings = tokenizer(join_all(dataset["test"]["post_tokens"]), padding=True, truncation=True, return_tensors="pt")

In [14]:
len(dataset["train"])

15383

In [15]:
class HateModel(torch.nn.Module):
  def __init__(self, bert):
    super(HateModel, self).__init__()
    self.bert = bert
    self.linear1 = torch.nn.Linear(2, 128)
    self.linear2 = torch.nn.Linear(128, 3)

  def forward(self, x, attention_mask, labels):
    x = self.bert(x, attention_mask=attention_mask, labels=labels)
    print(x)
    x = F.relu(self.linear1(x[1]))
    x = F.softmax(self.linear2(x))
    return x

In [16]:
class HateExplain(torch.utils.data.Dataset):
  def __init__(self, encodings, dataset):
    self.dataset = dataset
    self.encodings = encodings


  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = self.get_label(self.dataset[idx]["annotators"]["label"])
    return item

  def __len__(self):
    return len(self.dataset)

  def get_label(self, arr):
    cnt = [0, 0, 0]
    for v in arr:
      cnt[v] += 1
    
    # 0 - hate
    # 1 - normal
    # 2 - offensive
    return 1 if torch.argmax(torch.as_tensor(cnt)) != 1 else 0

      

train_dataset = HateExplain(train_encodings, dataset["train"])
val_dataset = HateExplain(val_encodings, dataset["validation"])
test_dataset = HateExplain(test_encodings, dataset["test"])

In [17]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
from transformers import AdamW
from torch.nn import functional as F
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
# device = torch.device('cpu')

model.to(device)
# hate_model = HateModel(model)
# hate_model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)

EPOCH = 400

for epoch in range(EPOCH):
  model.train(True)
  y_true = torch.as_tensor([]).to(device)
  scores = torch.as_tensor([]).to(device)
  cnt = 0
  tot_loss = 0
  for batch in train_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    # loss = criterion(y_pred, labels)
    loss = outputs[0]

    y_pred = F.softmax(outputs[1], dim=1)
    score = y_pred
    scores = torch.cat([scores, score])
    y_true = torch.cat([y_true, labels])

    tot_loss += loss.item()
    cnt += 1

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  y_pred = torch.argmax(scores, dim=1)
  print("Training Epoch", epoch + 1)
  print("Micro f1:", f1_score(y_true.cpu(), y_pred.cpu(), average="micro"))
  print("Macro f1:", f1_score(y_true.cpu(), y_pred.cpu(), average="macro"))
  print("accuracy:", accuracy_score(y_true.cpu(), y_pred.cpu()))
  # print("ROC AUC score:", roc_auc_score(y_true.cpu(), scores.detach().cpu(), average="weighted", multi_class="ovr"))
  print("Loss:", tot_loss / cnt)

  with torch.no_grad():
    model.train(False)
    y_true = torch.as_tensor([]).to(device)
    scores = torch.as_tensor([]).to(device)
    cnt = 0
    tot_loss = 0
    for batch in val_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      # loss = criterion(y_pred, labels)
      loss = outputs[0]

      y_pred = F.softmax(outputs[1], dim=1)
      score = y_pred
      scores = torch.cat([scores, score])
      y_true = torch.cat([y_true, labels])
      
      tot_loss += loss.item()
      cnt += 1
  
    y_pred = torch.argmax(scores, dim=1)
    print("Validation Epoch", epoch + 1)
    print("Micro f1:", f1_score(y_true.cpu(), y_pred.cpu(), average="micro"))
    print("Macro f1:", f1_score(y_true.cpu(), y_pred.cpu(), average="macro"))
    print("accuracy:", accuracy_score(y_true.cpu(), y_pred.cpu()))
    # print("ROC AUC score:", roc_auc_score(y_true.cpu(), scores.detach().cpu(), average="weighted", multi_class="ovr"))
    print("Loss:", tot_loss / cnt)
    print("\n")


# model.eval()

cuda
Training Epoch 1
Micro f1: 0.6680751478905285
Macro f1: 0.6342036356393609
accuracy: 0.6680751478905285
Loss: 0.6090345782016766
Validation Epoch 1
Micro f1: 0.6810613943808532
Macro f1: 0.6531747019995826
accuracy: 0.6810613943808532
Loss: 0.5919122476715687


Training Epoch 2
Micro f1: 0.6828316973282195
Macro f1: 0.6577286154390675
accuracy: 0.6828316973282195
Loss: 0.5902677984470637
Validation Epoch 2
Micro f1: 0.6966701352757544
Macro f1: 0.6630640464151959
accuracy: 0.6966701352757544
Loss: 0.578581365918325


Training Epoch 3
Micro f1: 0.6874471819541051
Macro f1: 0.6647938154758568
accuracy: 0.6874471819541051
Loss: 0.5871233382802495
Validation Epoch 3
Micro f1: 0.7065556711758585
Macro f1: 0.6888896542223446
accuracy: 0.7065556711758585
Loss: 0.5709110000901971


Training Epoch 4
Micro f1: 0.6895274003770396
Macro f1: 0.666391894497077
accuracy: 0.6895274003770396
Loss: 0.5844808244407796
Validation Epoch 4
Micro f1: 0.696149843912591
Macro f1: 0.6906619397070123
accura

In [ ]:
model.eval()

In [ ]:
tests = [
         "I love you", 
         "nigga i dont know you", 
         "muslims are the terrorist and they are the reason world is now full of shits", 
         "i love my country"
         ]
inputs = tokenizer(tests, truncation=True, padding=True, return_tensors="pt")
inputs.to(device)
outputs = model(**inputs)

In [ ]:
outputs[0]

In [ ]:
torch.argmax(torch.nn.Softmax(dim=1)(outputs[0]), dim=1)

In [ ]:
dataset["train"][0]